In [6]:
!pip install opencv-python numpy

In [7]:
import cv2
import numpy as np

# Load YOLOv3 model
def load_yolo_model(weights_path, cfg_path):
    net = cv2.dnn.readNet(weights_path, cfg_path)
    layer_names = net.getUnconnectedOutLayersNames()
    return net, layer_names

# Perform helmet detection
def detect_helmet(image, net, layer_names):
    height, width = image.shape[:2]

    # Prepare the image for YOLO model
    blob = cv2.dnn.blobFromImage(image, scalefactor=1/255.0, size=(416, 416), swapRB=True, crop=False)
    net.setInput(blob)
    detections = net.forward(layer_names)

    helmets = []
    for detection in detections:
        for obj in detection:
            scores = obj[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]

            if confidence > 0.5 and class_id == 0:  # Assuming class 0 is helmet
                center_x = int(obj[0] * width)
                center_y = int(obj[1] * height)
                w = int(obj[2] * width)
                h = int(obj[3] * height)

                # Get the coordinates for drawing the bounding box
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                helmets.append((x, y, w, h))

    return helmets

# Load YOLOv3 model
weights_path = "C:/Users/devis/detectionfiles/yolov3-helmet.weights"
cfg_path = "C:/Users/devis/detectionfiles/yolov3-helmet.cfg"
net, layer_names = load_yolo_model(weights_path, cfg_path)

# Load the image
image_path = "C:/Users/devis/detectionfiles/images/img2.jpg"
image = cv2.imread(image_path)

# Perform helmet detection
helmets = detect_helmet(image, net, layer_names)

# Draw bounding boxes around helmets
for x, y, w, h in helmets:
    cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)

# Display the result
cv2.imshow('Helmet Detection', image)
cv2.waitKey(0)
cv2.destroyAllWindows()
